In [1]:
import torch
import dlc_practical_prologue as prologue

In [2]:
mnist = prologue.load_data()

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [3]:
train_data_mnist = mnist[0]
train_label_mnist = mnist[1]
test_data_mnist = mnist[2]
test_label_mnist = mnist[3]

In [4]:
def nearest_classification(train_input, train_label, x):
    l2_norms = torch.sum((train_input - x) ** 2, axis=1).view(-1)
    lowest_norm = torch.argmin(l2_norms)
    return train_label[lowest_norm].item()

In [5]:
nearest_classification(train_data_mnist, train_label_mnist, test_data_mnist[2])

1

In [11]:
def compute_nb_errors(train_input, train_target, test_input, test_target, mean=None, proj=None):
    train_input = train_input.clone()
    test_input = test_input.clone()
    if mean is not None:
        train_input = train_input - mean
        test_input = test_input - mean
    
    if proj is not None:
        train_input = train_input @ proj
        test_input = test_input @ proj
    
    test_labels = []
    for i in range(test_input.shape[0]):
        test_labels.append(nearest_classification(train_input, train_target, test_input[i]))
    
    return torch.sum(test_target != torch.tensor(test_labels)).item()

In [12]:
compute_nb_errors(train_data_mnist, train_label_mnist, test_data_mnist, test_label_mnist)

172

In [15]:
def PCA(x):
    mean = torch.mean(x, axis=0)
    x = x - mean
    eig_values, eig_vectors = torch.eig(x.t() @ x, eigenvectors=True)
    _, indices = torch.sort(eig_values, axis=0, descending=True)
    return mean, eig_vectors[indices[:,0]]

In [16]:
mean_mnist, eig_vec_mnist = PCA(train_data_mnist)

In [21]:
pca_basis_mnist = eig_vec_mnist[:,:2]
pca_basis_mnist.shape

torch.Size([784, 2])

In [23]:
compute_nb_errors(train_data_mnist, train_label_mnist, test_data_mnist, test_label_mnist, mean=mean_mnist,\
                  proj=pca_basis_mnist @ pca_basis_mnist.t())

643

In [24]:
def tries(train_input, train_target, test_input, test_target, mean=None, proj=None, projections=None):
    for p in projections:
        pca_basis_mnist = eig_vec_mnist[:,:p]
        print(compute_nb_errors(train_data_mnist, train_label_mnist, test_data_mnist, test_label_mnist, mean=mean_mnist,\
                          proj=pca_basis_mnist @ pca_basis_mnist.t()))

In [26]:
tries(train_data_mnist, train_label_mnist, test_data_mnist, test_label_mnist, mean=mean_mnist,\
                  proj=pca_basis_mnist @ pca_basis_mnist.t(), projections=[2,30,50,100,500,1000])

643
187
177
168
156
170
